# 3. Modelo B

Basado en PCA y clusters:
- PCA
- Clusterización en 3 grupos
- Gridsearch de LGBM para cada grupo

## Librerías

In [ ]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score, \
roc_curve, roc_auc_score, ConfusionMatrixDisplay, multilabel_confusion_matrix

# Otros
import warnings
warnings.filterwarnings('ignore')

## Carga de datos

In [ ]:
df = pd.read_csv(r'..\data\processed\train_2.csv').drop(columns='id')
df.head()

## Tratamiento
Lo mismo que en notebook 2 pero sin log ni escalado

In [ ]:
# OHE
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', dtype=int)

encoded_mtrans = pd.DataFrame(ohe.fit_transform(df[['mtrans']]),
                             columns=ohe.get_feature_names_out(['mtrans']),
                             index=df.index)

# Elimino una de las columnas para mejorar el rendimiento
encoded_mtrans.drop(columns='mtrans_physic', inplace=True)

# Concateno
df = pd.concat([df, encoded_mtrans], axis=1).drop(columns='mtrans')



# MAPEO
df.caec = df.caec.map({'no':0,
                     'Sometimes':1,
                     'Frequently':2,
                     'Always':3})
df.calc = df.calc.map({'no':0,
                     'Sometimes':1,
                     'Frequently':2})
df.nobeyesdad = df.nobeyesdad.map({'Insufficient_Weight':0,
           'Normal_Weight':1,
           'Overweight_Level_I':2,
           'Overweight_Level_II':3,
           'Obesity_Type_I':4,
           'Obesity_Type_II':5,
           'Obesity_Type_III':6})


# SELECCIÓN COLUMNAS
df = df[['age', 'height', 'weight', 'family_history_with_overweight', 'favc', 'fcvc', 'caec', 'ch2o', 'faf', 'nobeyesdad']]

## PCA

In [ ]:
# Separo la target
X = df.drop(columns='nobeyesdad')
y = df['nobeyesdad']

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

print('Varianza explicada por cada dimensión:', pca.explained_variance_ratio_)
print('Varianza explicada total:', pca.explained_variance_ratio_.sum())

## Clustering
Vamos a clusterizar en 3 grupos. Idealmente, la división sería:
- C1: insufficient weight y normal weight
- C2: overweight_I, overweight_II
- C3: obesity_type_I, obesity_type_II, obesity_type_III

Evidentemente, no podemos asegurar que el algoritmo vaya a realizar esta división. Se adjunta esta información sólo para justificar el porqué de los 3 grupos.